# Data Extraction and Analysis Tutorial

This notebook demonstrates the basic functionality of the DSS Pollution Extraction package, developed at the Alfred Wegener Institute (AWI). 

The package provides comprehensive tools for:
- Loading and exploring pollution data from NetCDF files
- Temporal and spatial data extraction
- Basic visualization and analysis
- Data export in multiple formats

## Supported Pollutants
- **Black Carbon (BC)**: Aerosol Optical Depth
- **Nitrogen Dioxide (NO₂)**: Surface concentration
- **PM₂.₅**: Fine particulate matter
- **PM₁₀**: Coarse particulate matter

## Prerequisites
Make sure you have the required packages installed:
```bash
pip install pollution-extraction
```

In [ ]:
# Import required libraries
import warnings
from pathlib import Path

import matplotlib.pyplot as plt

# Scientific computing libraries
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr

# Geospatial libraries
# DSS Pollution Extraction package
from pollution_extraction import PollutionAnalyzer

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set paths
DATA_DIR = Path('../data')
OUTPUT_DIR = Path('./output')
OUTPUT_DIR.mkdir(exist_ok=True)

print("✅ All libraries imported successfully!")
print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

## 1. Loading and Exploring Pollution Data

Let's start by loading a sample PM2.5 dataset and exploring its structure.

In [ ]:
# Load sample PM2.5 data
sample_file = DATA_DIR / 'sample_pm25.nc'

# Check if sample file exists
if sample_file.exists():
    print(f"Loading data from: {sample_file}")

    # Initialize the analyzer
    analyzer = PollutionAnalyzer(sample_file, pollution_type='pm25')

    # Print basic dataset information
    analyzer.print_summary()

else:
    print(f"Sample file not found: {sample_file}")
    print("Please ensure you have the sample data file in the examples/data directory.")

    # Create a simple example dataset for demonstration
    print("\nCreating example dataset for demonstration...")

    # Generate sample data (this is just for demo purposes)
    time = pd.date_range('2023-01-01', '2023-12-31', freq='D')
    x = np.linspace(4000000, 5000000, 100)  # LAEA projection coordinates
    y = np.linspace(3000000, 4000000, 80)

    # Create sample PM2.5 data with some realistic patterns
    np.random.seed(42)
    data = np.random.exponential(15, (len(time), len(y), len(x)))

    # Add seasonal variation
    seasonal_factor = 1 + 0.3 * np.sin(2 * np.pi * np.arange(len(time)) / 365.25)
    data = data * seasonal_factor[:, np.newaxis, np.newaxis]

    # Create dataset
    sample_ds = xr.Dataset(
        {
            'PM2p5_downscaled': (['time', 'y', 'x'], data)
        },
        coords={
            'time': time,
            'x': x,
            'y': y
        }
    )

    # Add attributes
    sample_ds['PM2p5_downscaled'].attrs = {
        'units': 'μg/m³',
        'long_name': 'PM2.5 mass concentration',
        'standard_name': 'mass_concentration_of_pm2p5_ambient_aerosol_particles_in_air'
    }

    # Save sample dataset
    sample_ds.to_netcdf(sample_file)
    print(f"✅ Sample dataset created and saved to: {sample_file}")

    # Now load it with the analyzer
    analyzer = PollutionAnalyzer(sample_file, pollution_type='pm25')
    analyzer.print_summary()

In [ ]:
# Get detailed information about the dataset
info = analyzer.get_info()

print("📊 Dataset Information:")
print(f"  Time range: {info['basic_info']['time_range']}")
print(f"  Spatial extent: {info['basic_info']['spatial_extent']}")
print(f"  Total time steps: {info['basic_info']['total_time_steps']}")
print(f"  Grid shape: {info['basic_info']['grid_shape']}")

print("\n📈 Data Statistics:")
stats = info['data_summary']
for key, value in stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.3f} μg/m³")
    else:
        print(f"  {key}: {value}")

# Show the raw dataset structure
print("\n🔍 Raw Dataset Structure:")
print(analyzer.dataset)

## 2. Basic Visualization

Let's create some basic visualizations to understand the data patterns.

In [ ]:
# Create a spatial map for the first time step
fig = analyzer.plot_map(
    time_index=0,
    title='PM2.5 Spatial Distribution (First Day)',
    figsize=(12, 8),
    save_path=OUTPUT_DIR / 'pm25_spatial_day1.png'
)
plt.show()

# Create a spatial map for mid-year
fig = analyzer.plot_map(
    time_index=180,  # Around day 180 (July)
    title='PM2.5 Spatial Distribution (Mid-Year)',
    figsize=(12, 8),
    save_path=OUTPUT_DIR / 'pm25_spatial_midyear.png'
)
plt.show()

print("✅ Spatial maps created and saved!")

In [ ]:
# Create time series plot (domain average)
fig = analyzer.plot_time_series(
    title='PM2.5 Time Series (Domain Average)',
    figsize=(14, 6),
    save_path=OUTPUT_DIR / 'pm25_timeseries.png'
)
plt.show()

# Create seasonal cycle plot
fig = analyzer.plot_seasonal_cycle(
    title='PM2.5 Seasonal Cycle',
    figsize=(10, 6),
    save_path=OUTPUT_DIR / 'pm25_seasonal.png'
)
plt.show()

print("✅ Time series plots created and saved!")

## 3. Temporal Analysis

Now let's perform various temporal aggregations to understand pollution patterns over time.

In [ ]:
# Monthly averages
print("🗓️ Calculating monthly averages...")
monthly_avg = analyzer.get_monthly_averages()
print(f"Monthly data shape: {monthly_avg.dims}")
print(f"Available months: {monthly_avg.time.dt.month.values}")

# Annual average
print("\n📅 Calculating annual average...")
annual_avg = analyzer.get_annual_averages()
print(f"Annual average shape: {annual_avg.dims}")

# Seasonal averages
print("\n🌱 Calculating seasonal averages...")
seasonal_avg = analyzer.get_seasonal_averages()
print(f"Seasonal data shape: {seasonal_avg.dims}")
print(f"Available seasons: {seasonal_avg.season.values}")

# Display seasonal statistics
print("\n📊 Seasonal Statistics:")
for season in seasonal_avg.season.values:
    season_data = seasonal_avg.sel(season=season)
    mean_val = float(season_data.PM2p5_downscaled.mean())
    print(f"  {season}: {mean_val:.2f} μg/m³")

In [ ]:
# Visualize monthly trends
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('PM2.5 Temporal Analysis', fontsize=16, fontweight='bold')

# Monthly average map
monthly_jan = monthly_avg.isel(time=0)  # January
im1 = axes[0,0].imshow(monthly_jan.PM2p5_downscaled.values,
                       cmap='Reds', aspect='auto')
axes[0,0].set_title('January Average')
axes[0,0].set_xlabel('X coordinate')
axes[0,0].set_ylabel('Y coordinate')
plt.colorbar(im1, ax=axes[0,0], label='μg/m³')

# July average map
monthly_jul = monthly_avg.isel(time=6)  # July
im2 = axes[0,1].imshow(monthly_jul.PM2p5_downscaled.values,
                       cmap='Reds', aspect='auto')
axes[0,1].set_title('July Average')
axes[0,1].set_xlabel('X coordinate')
axes[0,1].set_ylabel('Y coordinate')
plt.colorbar(im2, ax=axes[0,1], label='μg/m³')

# Monthly time series
monthly_means = monthly_avg.PM2p5_downscaled.mean(dim=['x', 'y'])
axes[1,0].plot(monthly_means.time.dt.month, monthly_means.values,
               marker='o', linewidth=2, markersize=6)
axes[1,0].set_title('Monthly Domain Averages')
axes[1,0].set_xlabel('Month')
axes[1,0].set_ylabel('PM2.5 (μg/m³)')
axes[1,0].grid(True, alpha=0.3)
axes[1,0].set_xticks(range(1, 13))

# Seasonal comparison
seasonal_means = seasonal_avg.PM2p5_downscaled.mean(dim=['x', 'y'])
axes[1,1].bar(seasonal_means.season.values, seasonal_means.values,
              color=['skyblue', 'lightgreen', 'orange', 'lightcoral'])
axes[1,1].set_title('Seasonal Domain Averages')
axes[1,1].set_xlabel('Season')
axes[1,1].set_ylabel('PM2.5 (μg/m³)')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'pm25_temporal_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Temporal analysis visualization created!")

## 4. Spatial Data Extraction

Now let's extract pollution data at specific locations and regions.

In [ ]:
# Define some example monitoring station locations
# Using LAEA projection coordinates (typical for European data)
monitoring_stations = [
    (4321000, 3210000),  # Station 1
    (4450000, 3350000),  # Station 2
    (4580000, 3150000),  # Station 3
    (4200000, 3400000),  # Station 4
]

print(f"📍 Extracting data at {len(monitoring_stations)} monitoring stations...")

# Extract data at points
point_data = analyzer.extract_at_points(monitoring_stations)
print(f"Extracted data shape: {point_data.dims}")
print(f"Available variables: {list(point_data.data_vars)}")

# Display point extraction results
print("\n📊 Point Extraction Summary:")
for i, (x, y) in enumerate(monitoring_stations):
    station_data = point_data.isel(location=i)
    mean_conc = float(station_data.PM2p5_downscaled.mean())
    max_conc = float(station_data.PM2p5_downscaled.max())
    min_conc = float(station_data.PM2p5_downscaled.min())

    print(f"  Station {i+1} ({x}, {y}):")
    print(f"    Mean: {mean_conc:.2f} μg/m³")
    print(f"    Max:  {max_conc:.2f} μg/m³")
    print(f"    Min:  {min_conc:.2f} μg/m³")

In [ ]:
# Visualize time series for each monitoring station
fig, ax = plt.subplots(figsize=(14, 8))

colors = ['blue', 'red', 'green', 'orange']

for i in range(len(monitoring_stations)):
    station_data = point_data.isel(location=i)
    ax.plot(station_data.time, station_data.PM2p5_downscaled,
            label=f'Station {i+1}', color=colors[i], linewidth=1.5, alpha=0.8)

ax.set_title('PM2.5 Time Series at Monitoring Stations', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('PM2.5 Concentration (μg/m³)')
ax.legend()
ax.grid(True, alpha=0.3)

# Format x-axis
import matplotlib.dates as mdates

ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'pm25_stations_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Station time series plot created!")

## 5. Data Export

Let's export our analysis results in various formats for further use.

In [ ]:
# Export annual average to GeoTIFF
print("💾 Exporting annual average to GeoTIFF...")
geotiff_path = OUTPUT_DIR / 'pm25_annual_average.tif'
analyzer.export_to_geotiff(
    str(geotiff_path),
    aggregation_method='mean'
)
print(f"✅ GeoTIFF exported to: {geotiff_path}")

# Export point data to CSV
print("\n💾 Exporting point data to CSV...")
analyzer.exporter.extracted_points_to_formats(
    point_data,
    output_dir=OUTPUT_DIR,
    formats=['csv'],
    base_filename='pm25_monitoring_stations'
)
print(f"✅ Point data exported to: {OUTPUT_DIR}")

# Export monthly averages to NetCDF
print("\n💾 Exporting monthly averages to NetCDF...")
netcdf_path = OUTPUT_DIR / 'pm25_monthly_averages.nc'
monthly_avg.to_netcdf(netcdf_path)
print(f"✅ Monthly averages exported to: {netcdf_path}")

# Create metadata file
print("\n📋 Creating metadata file...")
metadata_path = OUTPUT_DIR / 'analysis_metadata.json'
analyzer.exporter.create_metadata_file(
    metadata_path,
    processing_info={
        'analysis_type': 'basic_extraction_tutorial',
        'processing_date': pd.Timestamp.now().isoformat(),
        'analyst': 'DSS Pollution Extraction Tutorial',
        'description': 'Basic data extraction and analysis example'
    }
)
print(f"✅ Metadata exported to: {metadata_path}")

## 6. Summary and Next Steps

### What We've Accomplished

In this tutorial, we've demonstrated the core functionality of the DSS Pollution Extraction package:

1. **✅ Data Loading**: Loaded PM2.5 pollution data from NetCDF format
2. **✅ Data Exploration**: Examined dataset structure and basic statistics
3. **✅ Visualization**: Created spatial maps and time series plots
4. **✅ Temporal Analysis**: Calculated monthly, seasonal, and annual averages
5. **✅ Spatial Extraction**: Extracted data at specific monitoring stations
6. **✅ Data Export**: Exported results in multiple formats (GeoTIFF, CSV, NetCDF)

### Key Features Demonstrated

- **Multi-format Support**: NetCDF input, GeoTIFF/CSV/NetCDF output
- **Temporal Aggregations**: Monthly, seasonal, annual averages
- **Spatial Extraction**: Point-based data extraction
- **Visualization**: Maps, time series, seasonal cycles
- **Metadata Management**: Automatic metadata generation

### Next Steps

Explore the other example notebooks:

1. **`advanced_spatial_analysis.ipynb`**: Advanced spatial analysis techniques
2. **`pollution_analysis_tutorial.ipynb`**: Complete comprehensive tutorial
3. **`temporal_pattern_analysis.ipynb`**: Advanced temporal pattern analysis

### Additional Resources

- 📚 [Package Documentation](https://dss-pollution-extraction.readthedocs.io/)
- 🐛 [Report Issues](https://github.com/MuhammadShafeeque/dss-pollution-extraction/issues)
- 💬 [Discussions](https://github.com/MuhammadShafeeque/dss-pollution-extraction/discussions)
- 📧 Contact: muhammad.shafeeque@awi.de

In [ ]:
# Clean up and close the analyzer
analyzer.close()

# Display output files
print("🎯 Tutorial completed successfully!")
print(f"\n📁 Output files created in: {OUTPUT_DIR}")
for file_path in OUTPUT_DIR.glob('*'):
    print(f"  - {file_path.name}")

print("\n🚀 Ready to explore more advanced features in the other notebooks!")